In [1]:
import os
import gc
import time
import math
import datetime
from math import log, floor
from sklearn.neighbors import KDTree

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.utils import shuffle
from tqdm.notebook import tqdm as tqdm

import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad

import scipy
import statsmodels
from scipy import signal
import statsmodels.api as sm
from fbprophet import Prophet
from scipy.signal import butter, deconvolve
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

import warnings
warnings.filterwarnings("ignore")

!pip install pyramid-arima
from pyramid.arima import auto_arima

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 604kB 2.8MB/s 


In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Kaggle_data/m5-forecasting-accuracy/sales_train_evaluation.csv')
calendar = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Kaggle_data/m5-forecasting-accuracy/calendar.csv')
sell_prices = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Kaggle_data/m5-forecasting-accuracy/sell_prices.csv')

In [7]:
event_type_list=list(set(list(calendar['event_type_1'].unique()) + list(calendar['event_type_2'].unique())))
print(event_type_list)
event_type_list = event_type_list[1:]
print(event_type_list)

[nan, 'Sporting', 'National', 'Religious', 'Cultural']
['Sporting', 'National', 'Religious', 'Cultural']


In [8]:
event_name_list= list(set(list(calendar['event_name_1'].unique()) + list(calendar['event_name_2'].unique())))
print(event_name_list)
event_name_list = event_name_list[1:]
print(event_name_list)

[nan, 'IndependenceDay', 'Chanukah End', 'Purim End', 'LentStart', 'NBAFinalsStart', "Mother's day", 'VeteransDay', 'OrthodoxChristmas', 'Cinco De Mayo', 'SuperBowl', 'Pesach End', "Father's day", 'ValentinesDay', 'Easter', 'Christmas', 'PresidentsDay', 'LentWeek2', 'NBAFinalsEnd', 'ColumbusDay', 'EidAlAdha', 'Thanksgiving', 'MartinLutherKingDay', 'MemorialDay', 'Ramadan starts', 'NewYear', 'StPatricksDay', 'LaborDay', 'OrthodoxEaster', 'Eid al-Fitr', 'Halloween']
['IndependenceDay', 'Chanukah End', 'Purim End', 'LentStart', 'NBAFinalsStart', "Mother's day", 'VeteransDay', 'OrthodoxChristmas', 'Cinco De Mayo', 'SuperBowl', 'Pesach End', "Father's day", 'ValentinesDay', 'Easter', 'Christmas', 'PresidentsDay', 'LentWeek2', 'NBAFinalsEnd', 'ColumbusDay', 'EidAlAdha', 'Thanksgiving', 'MartinLutherKingDay', 'MemorialDay', 'Ramadan starts', 'NewYear', 'StPatricksDay', 'LaborDay', 'OrthodoxEaster', 'Eid al-Fitr', 'Halloween']


In [0]:
sell_prices['id' ]= sell_prices['item_id']+'_'+sell_prices['store_id']
sell_prices.drop(['store_id','item_id'],axis=1,inplace=True)

---

In [10]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,3,0,1,1,1,3,0,1,1,0,0,0,2,0,3,5,0,0,1,1,0,2,1,2,2,1,0,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,2,2,1,2,1,1,1,0,1,1,1,0,0,1,1,0,2,1,0,0,0,0,2,1,3,0,0,1,0,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,0,1,0,5,4,1,0,1,3,7,2,0,0,1,2,4,1,6,4,0,0,0,2,2,4,2,1,1,1,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,2,1,1,0,1,1,2,2,2,4,1,0,2,3,1,0,3,2,3,1,1,3,2,3,2,2,2,2,0,0,0,2,1,0,0,2,1,0


In [11]:
sales_data=data.iloc[:,6:]
sales_data['id'] = data['id']

del data
gc.collect()

sales_data = sales_data.melt(id_vars='id')
sales_data.id.replace(regex='_validation',value='',inplace=True)
sales_data.rename({'variable':'d'},axis=1,inplace=True)

sales_data['d'].replace(regex ='d_',value='',inplace=True)
sales_data['d'] = sales_data['d'].astype('int16')
calendar['d'].replace(regex ='d_',value='',inplace=True)
calendar['d'] = calendar['d'].astype('int16')

sales_data = sales_data.merge(calendar[['d','wm_yr_wk']],on='d')

sales_data = sales_data.merge(sell_prices[['id','wm_yr_wk','sell_price']],on=['id','wm_yr_wk'],how='left')

# sales_data['state'] = sales_data.id.apply(lambda x: x[-4:-2])
# sales_data['market'] = sales_data.id.apply(lambda x: x[-4:])

sales_data.head()

,id,d,value,wm_yr_wk,sell_price
0,HOBBIES_1_001_CA_1_evaluation,1,0,11101,NaN
1,HOBBIES_1_002_CA_1_evaluation,1,0,11101,NaN
2,HOBBIES_1_003_CA_1_evaluation,1,0,11101,NaN
3,HOBBIES_1_004_CA_1_evaluation,1,0,11101,NaN
4,HOBBIES_1_005_CA_1_evaluation,1,0,11101,NaN


In [0]:
def make_onehot_type_col(col):
  sales_data[col] = 0
  for day in calendar.loc[(calendar['event_type_1'] == col)|(calendar['event_type_2'] == col),'d'].values:
    sales_data.loc[sales_data['d']==day, col] +=1

def make_onehot_name_col(col):
  sales_data[col] = 0
  for day in calendar.loc[(calendar['event_name_1'] == col)|(calendar['event_name_2'] == col),'d'].values:
    sales_data.loc[sales_data['d']==day, col] +=1

In [0]:
for col in event_type_list:
  make_onehot_type_col(col)

In [14]:
sales_data.head()

,id,d,value,wm_yr_wk,sell_price,Sporting,National,Religious,Cultural
0,HOBBIES_1_001_CA_1_evaluation,1,0,11101,NaN,0,0,0,0
1,HOBBIES_1_002_CA_1_evaluation,1,0,11101,NaN,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,1,0,11101,NaN,0,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,1,0,11101,NaN,0,0,0,0
4,HOBBIES_1_005_CA_1_evaluation,1,0,11101,NaN,0,0,0,0


우선 value랑 days만 다루면 되기 때문에 날짜만 합치고 일단 넘어가자.

In [15]:
sales_pivot = sales_data.pivot_table(index='d',columns='id',values='value')
sales_pivot.head()

id,FOODS_1_001_CA_1_evaluation,FOODS_1_001_CA_2_evaluation,FOODS_1_001_CA_3_evaluation,FOODS_1_001_CA_4_evaluation,FOODS_1_001_TX_1_evaluation,FOODS_1_001_TX_2_evaluation,FOODS_1_001_TX_3_evaluation,FOODS_1_001_WI_1_evaluation,FOODS_1_001_WI_2_evaluation,FOODS_1_001_WI_3_evaluation,FOODS_1_002_CA_1_evaluation,FOODS_1_002_CA_2_evaluation,FOODS_1_002_CA_3_evaluation,FOODS_1_002_CA_4_evaluation,FOODS_1_002_TX_1_evaluation,FOODS_1_002_TX_2_evaluation,FOODS_1_002_TX_3_evaluation,FOODS_1_002_WI_1_evaluation,FOODS_1_002_WI_2_evaluation,FOODS_1_002_WI_3_evaluation,FOODS_1_003_CA_1_evaluation,FOODS_1_003_CA_2_evaluation,FOODS_1_003_CA_3_evaluation,FOODS_1_003_CA_4_evaluation,FOODS_1_003_TX_1_evaluation,FOODS_1_003_TX_2_evaluation,FOODS_1_003_TX_3_evaluation,FOODS_1_003_WI_1_evaluation,FOODS_1_003_WI_2_evaluation,FOODS_1_003_WI_3_evaluation,FOODS_1_004_CA_1_evaluation,FOODS_1_004_CA_2_evaluation,FOODS_1_004_CA_3_evaluation,FOODS_1_004_CA_4_evaluation,FOODS_1_004_TX_1_evaluation,FOODS_1_004_TX_2_evaluation,FOODS_1_004_TX_3_evaluation,FOODS_1_004_WI_1_evaluation,FOODS_1_004_WI_2_evaluation,FOODS_1_004_WI_3_evaluation,...,HOUSEHOLD_2_513_CA_1_evaluation,HOUSEHOLD_2_513_CA_2_evaluation,HOUSEHOLD_2_513_CA_3_evaluation,HOUSEHOLD_2_513_CA_4_evaluation,HOUSEHOLD_2_513_TX_1_evaluation,HOUSEHOLD_2_513_TX_2_evaluation,HOUSEHOLD_2_513_TX_3_evaluation,HOUSEHOLD_2_513_WI_1_evaluation,HOUSEHOLD_2_513_WI_2_evaluation,HOUSEHOLD_2_513_WI_3_evaluation,HOUSEHOLD_2_514_CA_1_evaluation,HOUSEHOLD_2_514_CA_2_evaluation,HOUSEHOLD_2_514_CA_3_evaluation,HOUSEHOLD_2_514_CA_4_evaluation,HOUSEHOLD_2_514_TX_1_evaluation,HOUSEHOLD_2_514_TX_2_evaluation,HOUSEHOLD_2_514_TX_3_evaluation,HOUSEHOLD_2_514_WI_1_evaluation,HOUSEHOLD_2_514_WI_2_evaluation,HOUSEHOLD_2_514_WI_3_evaluation,HOUSEHOLD_2_515_CA_1_evaluation,HOUSEHOLD_2_515_CA_2_evaluation,HOUSEHOLD_2_515_CA_3_evaluation,HOUSEHOLD_2_515_CA_4_evaluation,HOUSEHOLD_2_515_TX_1_evaluation,HOUSEHOLD_2_515_TX_2_evaluation,HOUSEHOLD_2_515_TX_3_evaluation,HOUSEHOLD_2_515_WI_1_evaluation,HOUSEHOLD_2_515_WI_2_evaluation,HOUSEHOLD_2_515_WI_3_evaluation,HOUSEHOLD_2_516_CA_1_evaluation,HOUSEHOLD_2_516_CA_2_evaluation,HOUSEHOLD_2_516_CA_3_evaluation,HOUSEHOLD_2_516_CA_4_evaluation,HOUSEHOLD_2_516_TX_1_evaluation,HOUSEHOLD_2_516_TX_2_evaluation,HOUSEHOLD_2_516_TX_3_evaluation,HOUSEHOLD_2_516_WI_1_evaluation,HOUSEHOLD_2_516_WI_2_evaluation,HOUSEHOLD_2_516_WI_3_evaluation
d,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3,2,1,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,1,0,0,8,1,0,0,0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
2,0,0,2,1,1,0,0,2,0,0,1,1,0,1,0,0,0,1,0,1,0,2,1,1,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,1,2,2,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1
3,0,0,1,1,1,1,0,0,0,0,0,1,1,2,0,1,1,0,0,1,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,1,1,0,2,0,1,0,0,1,0,2,1,0,0,0,0,0,0,0,1,3,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1
5,4,1,1,1,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [23]:
sales_pivot.index = calendar.loc[:1940,'date']
sales_pivot.index = pd.to_datetime(sales_pivot.index)
sales_pivot.head()

id,FOODS_1_001_CA_1_evaluation,FOODS_1_001_CA_2_evaluation,FOODS_1_001_CA_3_evaluation,FOODS_1_001_CA_4_evaluation,FOODS_1_001_TX_1_evaluation,FOODS_1_001_TX_2_evaluation,FOODS_1_001_TX_3_evaluation,FOODS_1_001_WI_1_evaluation,FOODS_1_001_WI_2_evaluation,FOODS_1_001_WI_3_evaluation,FOODS_1_002_CA_1_evaluation,FOODS_1_002_CA_2_evaluation,FOODS_1_002_CA_3_evaluation,FOODS_1_002_CA_4_evaluation,FOODS_1_002_TX_1_evaluation,FOODS_1_002_TX_2_evaluation,FOODS_1_002_TX_3_evaluation,FOODS_1_002_WI_1_evaluation,FOODS_1_002_WI_2_evaluation,FOODS_1_002_WI_3_evaluation,FOODS_1_003_CA_1_evaluation,FOODS_1_003_CA_2_evaluation,FOODS_1_003_CA_3_evaluation,FOODS_1_003_CA_4_evaluation,FOODS_1_003_TX_1_evaluation,FOODS_1_003_TX_2_evaluation,FOODS_1_003_TX_3_evaluation,FOODS_1_003_WI_1_evaluation,FOODS_1_003_WI_2_evaluation,FOODS_1_003_WI_3_evaluation,FOODS_1_004_CA_1_evaluation,FOODS_1_004_CA_2_evaluation,FOODS_1_004_CA_3_evaluation,FOODS_1_004_CA_4_evaluation,FOODS_1_004_TX_1_evaluation,FOODS_1_004_TX_2_evaluation,FOODS_1_004_TX_3_evaluation,FOODS_1_004_WI_1_evaluation,FOODS_1_004_WI_2_evaluation,FOODS_1_004_WI_3_evaluation,...,HOUSEHOLD_2_513_CA_1_evaluation,HOUSEHOLD_2_513_CA_2_evaluation,HOUSEHOLD_2_513_CA_3_evaluation,HOUSEHOLD_2_513_CA_4_evaluation,HOUSEHOLD_2_513_TX_1_evaluation,HOUSEHOLD_2_513_TX_2_evaluation,HOUSEHOLD_2_513_TX_3_evaluation,HOUSEHOLD_2_513_WI_1_evaluation,HOUSEHOLD_2_513_WI_2_evaluation,HOUSEHOLD_2_513_WI_3_evaluation,HOUSEHOLD_2_514_CA_1_evaluation,HOUSEHOLD_2_514_CA_2_evaluation,HOUSEHOLD_2_514_CA_3_evaluation,HOUSEHOLD_2_514_CA_4_evaluation,HOUSEHOLD_2_514_TX_1_evaluation,HOUSEHOLD_2_514_TX_2_evaluation,HOUSEHOLD_2_514_TX_3_evaluation,HOUSEHOLD_2_514_WI_1_evaluation,HOUSEHOLD_2_514_WI_2_evaluation,HOUSEHOLD_2_514_WI_3_evaluation,HOUSEHOLD_2_515_CA_1_evaluation,HOUSEHOLD_2_515_CA_2_evaluation,HOUSEHOLD_2_515_CA_3_evaluation,HOUSEHOLD_2_515_CA_4_evaluation,HOUSEHOLD_2_515_TX_1_evaluation,HOUSEHOLD_2_515_TX_2_evaluation,HOUSEHOLD_2_515_TX_3_evaluation,HOUSEHOLD_2_515_WI_1_evaluation,HOUSEHOLD_2_515_WI_2_evaluation,HOUSEHOLD_2_515_WI_3_evaluation,HOUSEHOLD_2_516_CA_1_evaluation,HOUSEHOLD_2_516_CA_2_evaluation,HOUSEHOLD_2_516_CA_3_evaluation,HOUSEHOLD_2_516_CA_4_evaluation,HOUSEHOLD_2_516_TX_1_evaluation,HOUSEHOLD_2_516_TX_2_evaluation,HOUSEHOLD_2_516_TX_3_evaluation,HOUSEHOLD_2_516_WI_1_evaluation,HOUSEHOLD_2_516_WI_2_evaluation,HOUSEHOLD_2_516_WI_3_evaluation
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-01-29,3,2,1,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,1,0,0,8,1,0,0,0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
2011-01-30,0,0,2,1,1,0,0,2,0,0,1,1,0,1,0,0,0,1,0,1,0,2,1,1,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,1,2,2,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1
2011-01-31,0,0,1,1,1,1,0,0,0,0,0,1,1,2,0,1,1,0,0,1,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2011-02-01,1,0,1,1,0,2,0,1,0,0,1,0,2,1,0,0,0,0,0,0,0,1,3,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1
2011-02-02,4,1,1,1,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [0]:
sales_pivot.to_csv('/content/drive/My Drive/Colab Notebooks/data/Kaggle_data/m5-forecasting-accuracy/sales_pivot.csv')